<a href="https://colab.research.google.com/github/jesus-acm/crimeZone-python-v1/blob/master/3_Selecci%C3%B3n_de_delitos_eje_en_cada_categor%C3%ADa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Aqui vamos a seleccionar nuestros delitos principales en cada eje y además obtener las colonias más peligrosas
## Posterior a esto vamos a calcular la relación de los delitos para ver si los podemos incluir en las categorías para finalmente
## Poder escoger datos que nos sean útiles en nuestro programa
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
!pip install hdbscan
%cd 'drive/My Drive/Colab_Notebooks_Crimezone/CrimeZone'

     |████████████████████████████████| 4.7MB 90kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.26-cp36-cp36m-linux_x86_64.whl size=2307225 sha256=30d52e65b93f9b35af547a1c1633e627e9cbf314e4ea86b3db7aa7095d1056b4
  Stored in directory: /root/.cache/pip/wheels/82/38/41/372f034d8abd271ef7787a681e0a47fc05d472683a7eb088ed
Successfully built hdbscan
/content/drive/.shortcut-targets-by-id/1s1XGSgZeOGfu83xeE4RiHvWK3TrYg3vT/Colab_Notebooks_Crimezone/CrimeZone


In [ ]:
crimenes = pd.read_csv('../CrimeZone_db/crimenes-GAM-limpios.csv')
crimenes.head()

,fecha,crimen,categoria,colonia,long,lat
0,2016-05-18 17:45:00,ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA,ROBO DE CELULAR CON VIOLENCIA,DEFENSORES DE LA REPÚBLICA,-99.141133,19.472193
1,2016-05-19 13:20:00,ROBO A NEGOCIO SIN VIOLENCIA,ROBO A NEGOCIO SV,SANTIAGO ATZACOALCO,-99.097474,19.496379
2,2016-05-05 17:00:00,ROBO A NEGOCIO CON VIOLENCIA,ROBO A NEGOCIO CV,25 DE JULIO,-99.083278,19.495574
3,2016-05-19 13:35:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE CV,LA LAGUNA TICOMÁN,-99.132372,19.514183
4,2016-05-18 11:00:00,ROBO A CASA HABITACION SIN VIOLENCIA,ROBO A CASA HABITACION SV,SANTA ISABEL TOLA,-99.116308,19.494444


In [ ]:
crimenes.dtypes

fecha         object
crimen        object
categoria     object
colonia       object
long         float64
lat          float64
dtype: object

In [ ]:
## Convertimos los tipos de datos de las columnas
crimenes = crimenes.convert_dtypes()
crimenes['fecha'] = pd.to_datetime(
    crimenes['fecha'],
    errors = 'coerce',
    format = '%Y/%m/%d %H:%M:%S'
)
crimenes.dtypes

fecha        datetime64[ns]
crimen               string
categoria            string
colonia              string
long                float64
lat                 float64
dtype: object

In [ ]:
## Desplegamos las categorias de nuestro dataset
crimenes['categoria'].unique()

<StringArray>
[                'ROBO DE CELULAR CON VIOLENCIA',
                             'ROBO A NEGOCIO SV',
                             'ROBO A NEGOCIO CV',
                          'ROBO A TRANSEUNTE CV',
                     'ROBO A CASA HABITACION SV',
                           'ROBO DE VEHICULO SV',
                               'ROBO A VEHICULO',
                     'ROBO A CASA HABITACION CV',
                           'ROBO DE VEHICULO CV',
                              'HOMICIDIO DOLOSO',
                           'ROBO EN MICROBUS CV',
 'LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO',
                 'ROBO DE CELULAR SIN VIOLENCIA',
                                  'NARCOMENUDEO',
                              'ROBO EN METRO SV',
                      'ROBO A CUENTAHABIENTE CV',
                          'ROBO A REPARTIDOR CV',
                               'ROBO EN TAXI CV',
                          'ROBO A TRANSEUNTE SV',
                          'ROBO A RE

In [ ]:
crimenes['categoria'].value_counts()

ROBO A NEGOCIO SV                                4850
ROBO A TRANSEUNTE CV                             3768
ROBO A VEHICULO                                  3710
ROBO DE VEHICULO SV                              3268
ROBO DE VEHICULO CV                              2358
ROBO A CASA HABITACION SV                        2055
ROBO EN MICROBUS CV                              1353
ROBO A NEGOCIO CV                                1132
ROBO DE CELULAR CON VIOLENCIA                    1057
ROBO DE CELULAR SIN VIOLENCIA                     871
ROBO A REPARTIDOR CV                              777
HOMICIDIO DOLOSO                                  584
ROBO A TRANSEUNTE SV                              407
NARCOMENUDEO                                      386
LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO     348
ROBO EN METRO SV                                  228
VIOLACIÓN                                         207
ROBO A CASA HABITACION CV                         172
ROBO A CUENTAHABIENTE CV    

# Selección de crimenes eje

### En nuestra cuenta vemos que los delitos más importantes refiriendonos a delitos a PEATONES es  "ROBO A TRANSEUNTE CV", esta será la variable eje donde empezaremos a hacer nuestras pruebas para encontar similitud de prescencia de crimenes.

### En tanto a lo referente a los delitos de transporte público, como nuestro trabajo terminal está enfocado a el Servicio de transporte público Metro vamos a utilizar "ROBO EN METRO SV"

### Lo que sigue es clusterizar los delitos a asaltos a transeuntes y calcular la similitud con otros delitos



# Clusterización de delitos:

* ## HDBSCAN
Este algoritmo se implementará para ver los focos de concentración del delito "ROBO A TRANSEUNTE CV"


In [ ]:
## Eliminamos columnas que no serán de utilidad para la implementación de nuestro algoritmo :)
## Sacamos la columna crimen 
crimenes = crimenes.drop(['crimen'],axis=1)

In [ ]:
## preparamos nuestro dataframe para el algoritmo
peatones = crimenes.rename(columns={'categoria':'crimen'}).set_index(['crimen']).loc['ROBO A TRANSEUNTE CV'].reset_index()

In [ ]:
def clusterizar(df):
  import hdbscan
  X = df[['lat', 'long']].values
  rads = np.radians(X)
  clusterer = hdbscan.HDBSCAN(min_cluster_size=2, metric='haversine')
  predictions = clusterer.fit_predict(rads)
  df_clusterizado = pd.concat([df.reset_index(), 
                       pd.DataFrame({'Cluster':predictions})], 
                      axis=1)
  df_clusterizado.drop('index', axis=1, inplace=True)
  return df_clusterizado

In [ ]:
clustered = clusterizar(peatones)
clustered

,crimen,fecha,colonia,long,lat,Cluster
0,ROBO A TRANSEUNTE CV,2016-05-19 13:35:00,LA LAGUNA TICOMÁN,-99.132372,19.514183,446
1,ROBO A TRANSEUNTE CV,2016-05-19 15:20:00,JUAN GONZÁLEZ ROMERO,-99.090854,19.506820,948
2,ROBO A TRANSEUNTE CV,2016-05-20 18:10:00,FERROCARRILERA,-99.097314,19.479611,573
3,ROBO A TRANSEUNTE CV,2016-05-20 23:30:00,RESIDENCIAL ZACATENCO,-99.120436,19.495638,-1
4,ROBO A TRANSEUNTE CV,2016-05-21 12:20:00,VILLA GUSTAVO A. MADERO,-99.113828,19.484084,769
...,...,...,...,...,...,...
3763,ROBO A TRANSEUNTE CV,2018-11-28 22:13:00,ARAGÓN INGUARÁN,-99.106636,19.472753,1010
3764,ROBO A TRANSEUNTE CV,2018-11-29 08:50:00,VALLE DE MADERO,-99.140769,19.546944,274
3765,ROBO A TRANSEUNTE CV,2018-11-29 16:00:00,C.T.M. ATZACOALCO,-99.106199,19.509012,322
3766,ROBO A TRANSEUNTE CV,2018-11-27 18:30:00,AMPLIACIÓN CASAS ALEMÁN,-99.081636,19.476153,747


In [ ]:
clustered.sort_values(by=['Cluster'])

,crimen,fecha,colonia,long,lat,Cluster
403,ROBO A TRANSEUNTE CV,2018-05-03 15:00:00,PROVIDENCIA,-99.067127,19.484915,-1
3155,ROBO A TRANSEUNTE CV,2020-04-18 13:06:00,SAN JUAN DE ARAGÓN,-99.080519,19.464903,-1
461,ROBO A TRANSEUNTE CV,2017-12-27 14:00:00,AMPLIACIÓN PROVIDENCIA,-99.070896,19.475763,-1
1505,ROBO A TRANSEUNTE CV,2018-02-10 06:10:00,ESTANZUELA,-99.108573,19.490102,-1
3154,ROBO A TRANSEUNTE CV,2020-04-17 16:35:00,SAN JUAN DE ARAGÓN,-99.083594,19.471799,-1
...,...,...,...,...,...,...
1297,ROBO A TRANSEUNTE CV,2017-04-12 16:15:00,GUADALUPE TEPEYAC,-99.122037,19.467349,1154
2637,ROBO A TRANSEUNTE CV,2018-09-21 22:18:00,GUADALUPE TEPEYAC,-99.122063,19.467713,1154
2165,ROBO A TRANSEUNTE CV,2018-06-11 22:00:00,GUADALUPE TEPEYAC,-99.122096,19.467614,1154
3255,ROBO A TRANSEUNTE CV,2016-11-16 20:22:00,GUADALUPE TEPEYAC,-99.122418,19.467356,1155


In [ ]:
clustered['colonia'].unique()

<StringArray>
[      'LA LAGUNA TICOMÁN',    'JUAN GONZÁLEZ ROMERO',
          'FERROCARRILERA',   'RESIDENCIAL ZACATENCO',
 'VILLA GUSTAVO A. MADERO',               'LA JOYITA',
     'CUCHILLA DEL TESORO',              'DEL OBRERO',
     'TEPEYAC INSURGENTES',              'LINDAVISTA',
 ...
      'EX ESCUELA DE TIRO',            'EL TEPETATAL',
 'LA ESMERALDA 1A SECCIÓN',        'EL ARBOLILLO CTM',
   'HÉROES DE CHAPULTEPEC',     'PRADOS DE CUAUTEPEC',
   'PLANETARIO LINDAVISTA',        'EDUARDO MOLINA I',
             'TLACAELEL 2',                  'INDECO']
Length: 176, dtype: string

## Funcion mapea cluster
La función se encarga de mapear con su respectivo color cada cluster.
Recibe los siguientes datos la funcion: 
* El dataframe que se va a mapear.
* La columna con la cual se quiere comparar el siguiente arreglo.
* Arreglo con los valores que se quiere obtener.

Finalmente retorna el mapa con los marcadores de cada cluster y cada uno con su información respectiva.

In [ ]:
def mapeaCluster(datos, por, arreglo):
  import folium
  import time
  import random

  dataframe = clustered
  dataframe = dataframe.set_index(por).loc[arreglo].reset_index()

  centro_lat, centro_lon = 19.490560, -99.133424
  folium_map = folium.Map(location=[centro_lat, centro_lon], zoom_start=12, tiles='cartodb positron')

  id_clusters = dataframe['Cluster'].unique()
  for id in id_clusters:
    datos_aux = dataframe[dataframe['Cluster']==id]
    r = lambda: random.randint(0,255)
    color_marker = str(('#%02X%02X%02X' % (r(),r(),r())))
    for i,row in datos_aux.iterrows():
      anio = time.strptime(str(row.fecha), '%Y-%m-%d %H:%M:%S').tm_year
      popup_text = f"<b> Nombre: </b> {row.crimen} <br> <b> Anio: </b> {anio} <br> <b>Cluster: </b> {row.Cluster}"
      folium.CircleMarker(location=[row.lat, row.long], color=color_marker , radius=5, tooltip=popup_text, fill=True, fill_opacity=0.4).add_to(folium_map)

  return folium_map

### Experimento con colonia

Probamos la función quedandonos con ciertas colonias de la Gustavo A. Madero, las cuales son: 
* LA LAGUNA TICOMÁN
* CUCHILLA DEL TESORO
* DEL OBRERO

Obtenemos el mapa con todos los cluster que contiene la colonia.

In [ ]:
mapeaCluster(clustered, 'colonia', ['LA LAGUNA TICOMÁN','CUCHILLA DEL TESORO','DEL OBRERO'])

### Experimento por Cluster

Probamos la función por Cluster, le proporcionamos a la función el conjunto de id de los Cluster que queremos visualizar.

Obtenemos el mapa con los marcadores de los cluster que deseamos visualizar.

In [ ]:
mapeaCluster(clustered, 'Cluster', [1, 2 ,3 , 4])

In [ ]:
def mapeaClusterColonia(datos):
  import folium
  import time
  import random
  # Se indexa por colonias paraclusterizar y mapear
  df = clustered.set_index('colonia');
  for i,colonia in df:
      df_aux =  df.loc[colonia].reset_index()
  # Se instancia el mapa y se configura
  centro_lat, centro_lon = 19.490560, -99.133424
  folium_map = folium.Map(location=[centro_lat, centro_lon], zoom_start=12, tiles='cartodb positron')

  #Por cada cluster
  id_clusters = dataframe['Cluster'].unique()
  for id in id_clusters:
    datos_aux = dataframe[dataframe['Cluster']==id]
    r = lambda: random.randint(0,255)
    color_marker = str(('#%02X%02X%02X' % (r(),r(),r())))
    for i,row in datos_aux.iterrows():
      anio = time.strptime(str(row.fecha), '%Y-%m-%d %H:%M:%S').tm_year
      popup_text = f"<b> Nombre: </b> {row.crimen} <br> <b> Anio: </b> {anio} <br> <b>Cluster: </b> {row.Cluster}"
      folium.CircleMarker(location=[row.lat, row.long], color=color_marker , radius=5, tooltip=popup_text, fill=True, fill_opacity=0.4).add_to(folium_map)

  return folium_map